In [ ]:
import geopandas as gpd
INPUT_FILE = 'BedrockP.gpkg'
bedrock_data = gpd.read_file(INPUT_FILE)
bedrock_data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point

#filtering the rock_type column to contain only 'ultramafic' and 'granodioritic intrusive'
df = bedrock_data[bedrock_data['rock_type'].str.contains('ultramafic|granodioritic intrusive')]

In [ ]:
# (EDA left but commented out) Plotting polygons initially
'''
fig, ax = plt.subplots()
df1.plot(ax=ax, color='blue', alpha=0.5)  # Ultramafic
df2.plot(ax=ax, color='red', alpha=0.5)   # Granodioritic
'''

In [ ]:
#EDA exploring the polygon borders/geometries
'''
bedrock_data['boundary']= bedrock_data.boundary
poly1 = bedrock_data['boundary'][0]
poly2 = bedrock_data['boundary'][5]
d = poly1.boundary.distance(poly2.boundary)
print(d)
'''

In [ ]:
#(cont'd)
'''
df = bedrock_data[bedrock_data['rock_type'].str.contains('ultramafic|granodioritic intrusive')]
rock1 = df[df['rock_type'].str.contains('ultramafic')]
rock2 = df[df['rock_type'].str.contains('granodioritic intrusive')]
intersections = rock2[rock2['boundary'].intersects(rock1['boundary'])]
intersections
'''

In [ ]:
# create separate rock type df's
df1 = df[df['rock_type'].str.contains('ultramafic')]
df2 = df[df['rock_type'].str.contains('granodioritic intrusive')]

# for the power function that will determine heatmap intensity fall-off
power = 0.5

# the fall-off distance(10KM)
max_distance = 10 * 10**3  # 10 km in meters

# the max. likelihood of finding cobalt, never exactly equal 1
max_likelihood = 0.99

#find all overlapping andintersecting ultramafic/granodioritic polygons first (highest likelihood, 0 distance)
overlap_polygons = gpd.overlay(df1, df2, how='intersection')

#generate grid to hold distances
xmin, ymin, xmax, ymax = df.total_bounds
grid_res = 100
grid_x, grid_y = np.mgrid[xmin:xmax:grid_res, ymin:ymax:grid_res]

# high value that will give near zero likelihood (distance away from the ultramafic/granodioritic borders)
large_distance = 50 * 10**3  # 50 km in meters

#compute minimum distances from each grid point to each type of rock
intensity = np.full_like(grid_x, large_distance)
#can change large_distance to some big big number to avoid grid cutoff, but this substantially increases runtime
for i in range(grid_x.shape[0]):
    for j in range(grid_x.shape[1]):
        point = Point(grid_x[i, j], grid_y[i, j])
        min_distance_to_df1 = df1.geometry.distance(point).min()
        min_distance_to_df2 = df2.geometry.distance(point).min()
        #check if the point is within any overlapping area, if so assign it the highest likelihood
        if any(overlap_polygons.geometry.apply(lambda poly: poly.contains(point))):
            intensity[i, j] = 0
        # If not within an overlapping area, check if the point is near both types of rocks
        elif min_distance_to_df1 < max_distance and min_distance_to_df2 < max_distance:
            intensity[i, j] = min(min_distance_to_df1, min_distance_to_df2)

#normalize intensity
intensity_norm = (intensity - intensity.min()) / (intensity.max() - intensity.min())

#compute likelihood using power function
likelihood = max_likelihood * (1 - intensity_norm ** power)

#plotting
fig, ax = plt.subplots(figsize=(20, 20))
c = ax.pcolormesh(grid_x, grid_y, likelihood, cmap='viridis', shading='auto', alpha=0.5)
fig.colorbar(c, ax=ax)
df1.boundary.plot(ax=ax, color='blue', label='ultramafic')
df2.boundary.plot(ax=ax, color='red', label='granodioritic intrusive')
plt.legend()
plt.show()
